# ST-Stochastic descent

- using SGD on $C$, $\{X_m\}_{m=0}^{k+l}$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import glob, os, psutil, time

os.chdir('../core')
import ssm_scripts, ssm_fit
from utility import get_subpop_stats, gen_data
from SSID_Hankel_loss import run_bad, plot_slim, plot_outputs_l2_gradient_test
os.chdir('../dev')

#np.random.seed(0)

# define problem size
p, n, k, l, T = 100, 10, 3, 3, 500

# settings for fitting algorithm
batch_size, max_zip_size, max_iter = 1, 100, 100
a, b1, b2, e = 0.1, 0.9, 0.99, 1e-8
linearity, stable, sym_psd = 'False', False, False

# I/O matter
mmap, chunksize = True, np.min((p,2000))
data_path, save_file = '../fits/', 'test'
verbose=True

# create subpopulations
sub_pops = (np.arange(0,p), np.arange(0,p))

obs_idx, idx_grp, co_obs, _, _, _, Om, _, _ = \
    get_subpop_stats(sub_pops=sub_pops, p=p, verbose=False)

# draw system matrices 
print('(p,n,k+l,T) = ', (p,n,k+l,T), '\n')
nr = 2 # number of real eigenvalues
eig_m_r, eig_M_r, eig_m_c, eig_M_c = 0.8, 0.99, 0.8, 0.99
pars_true, x, y, Qs, idx_a, idx_b = gen_data(p,n,k,l,T, nr,
                                             eig_m_r, eig_M_r, 
                                             eig_m_c, eig_M_c,
                                             mmap, chunksize,
                                             data_path)

pars_init='default'        
np.savez(data_path + save_file, 
         pars_init=pars_init,
         pars_true=pars_true, 
         pars_est=None,
         sub_pops=sub_pops,
         p=p,n=n,T=T,k=k,l=l,
         idx_a=idx_a, idx_b=idx_b,
         x=x)          

print('\n')
print(psutil.virtual_memory())
print(psutil.swap_memory())


# Load test dynamic texture

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import glob, os

os.chdir('../core')
import SSID_Hankel_loss 
from utility import get_subpop_stats, draw_sys, gen_data, gen_pars
from SSID_Hankel_loss import run_bad, plot_outputs_l2_gradient_test, l2_bad_sis_setup
os.chdir('../dev')

import psutil
import time

from scipy.io import loadmat
data = loadmat('/home/mackelab/Desktop/Projects/Stitching/data/dynamic_textures/fire.mat')['data']
T = data.shape[0]
p = data.shape[1] * data.shape[2] // 64
#  reshape data
y = np.zeros((T, p))
for t in range(T):
    y[t,:] = np.ravel(np.mean(data[t,:,:,:],axis=2)[np.ix_(np.arange(0,data.shape[1],8),np.arange(0,data.shape[2],8))])


# set fitting parameters
k,l = 2,2
n = 27

# settings for fitting algorithm
batch_size, max_zip_size, max_iter = 1, 100, 100
a, b1, b2, e = 0.1, 0.9, 0.99, 1e-8
linearity, stable, sym_psd = 'False', False, False

# I/O matter
mmap, chunksize = False, np.min((p,2000))
data_path, save_file = '../fits/', 'test'
verbose=True


# create subpopulations
sub_pops = (np.arange(0,p), np.arange(0,p))
obs_idx, idx_grp, co_obs, _, _, _, Om, _, _ = \
    get_subpop_stats(sub_pops=sub_pops, p=p, verbose=False)

# generate (subsampled) covariance matrices
pa, pb = np.min((p,1000)), np.min((p,1000))
idx_a = np.sort(np.random.choice(p, pa, replace=False))
idx_b = np.sort(np.random.choice(p, pb, replace=False))    
Qs = []
for m in range(k+l):
    Qs.append(None)
    print('computing time-lagged covariance for lag ', str(m))
    if mmap:
        Q = np.memmap(data_path+'Qs_'+str(m), dtype=np.float, mode='w+', shape=(pa,pb))
    else:
        Q = np.empty((pa,pb))
    Q[:] = np.cov(y[m:m-(k+l),idx_a].T, y[:-(k+l),idx_b].T)[:pa,pb:]     
    if mmap:
        del Q
        Qs[m] = np.memmap(data_path+'Qs_'+str(m), dtype=np.float, mode='r', shape=(pa,pb))
    else:
        Qs[m] = Q           
        
    

# Start fitting

In [ ]:
%matplotlib inline

t = time.time()
pars_init, pars_est, traces = run_bad(k=k,l=l,n=n,y=y, Qs=Qs,Om=Om,idx_a=idx_a, idx_b=idx_b,
                                      sub_pops=sub_pops,idx_grp=idx_grp,co_obs=co_obs,obs_idx=obs_idx,
                                      linearity=linearity,stable=stable,init=pars_init,
                                      alpha=a,b1=b1,b2=b2,e=e,max_iter=max_iter,batch_size=batch_size,
                                      verbose=verbose, sym_psd=sym_psd, max_zip_size=max_zip_size)

print('fitting time was ', time.time() - t, 's')
print('\n')
print(psutil.virtual_memory())
print(psutil.swap_memory())

plot_slim(Qs,k,l,pars_est,idx_a,idx_b,traces,mmap,data_path)

if p <= 50:
    plot_outputs_l2_gradient_test(pars_true=pars_true, pars_init=pars_init, 
                                  pars_est=pars_est, k=k, l=l, Qs=Qs, 
                                       Om=Om, traces = traces, idx_a=idx_a, idx_b=idx_b,
                                       linearity=linearity, idx_grp = idx_grp, co_obs = co_obs, 
                                       if_flip = True, m = 0)

# Loading data (+intermediate results)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy.optimize import fmin_bfgs, check_grad
import glob, os

os.chdir('../core')
import SSID_Hankel_loss 
from utility import get_subpop_stats, draw_sys, gen_data
from SSID_Hankel_loss import run_bad, plot_outputs_l2_gradient_test, l2_bad_sis_setup
os.chdir('../dev')

import psutil
import time

# load ancient code for drawing from LDS ...
os.chdir('../../../../pyRRHDLDS/core')
import ssm_scripts
import ssm_fit
os.chdir('../../code_le_stitch/iterSSID/python/dev')


from scipy.io import savemat # store results for comparison with Matlab code   

os.chdir('../fits/')

mmap = True
data_path = '../fits/'

save_file = np.load('test.npz')
p,n,T,k,l = save_file['p'], save_file['n'], save_file['T'], save_file['k'], save_file['l']
pars_true = save_file['pars_true'].tolist()
pars_est, pars_init = save_file['pars_est'].tolist(), save_file['pars_init'].tolist()
idx_a, idx_b = save_file['idx_a'], save_file['idx_b']
pa, pb = len(idx_a), len(idx_b)
#pa, pb = p,p

Qs = []
for m in range(k+l):
    print('loading time-lagged covariance for lag ', str(m))
    Qs.append(np.memmap('Qs_'+str(m), dtype=np.float, mode='r', shape=(pa,pb)))

y = np.memmap('y', dtype=np.float, mode='r', shape=(T,p))


sub_pops = (np.arange(p), np.arange(p))
obs_idx, idx_grp, co_obs, _, _, _, Om, _, _ = \
    get_subpop_stats(sub_pops=sub_pops, p=p, verbose=False)

chunksize = 5000
max_zip_size = 5000

verbose=True
            
print('\n')
print(psutil.virtual_memory())
print(psutil.swap_memory())


# Additional turns

# just one more turn...